In [1]:
from sionna.rt import load_scene, PlanarArray, Receiver, Transmitter

import mitsuba as mi
import drjit as dr
import numpy as np
import torch

In [2]:
scene = load_scene(filename="cubes/cubes.xml")

In [3]:
mi_scene = scene.mi_scene

In [4]:
shapes = mi_scene.shapes_dr()

# Will throw an error if there is a non-mesh shape
shapes = mi.MeshPtr(shapes)

face_normals = [
    s.face_normal(dr.arange(dr.cuda.UInt, s.face_count())).torch().T for s in shapes
]
face_normals = torch.vstack(face_normals)

# Find the average between the 3 vertices of a triangle mesh
# This will be the source for ray intersection testing
face_averages = []

shape_offset = {}
offset_counter = 0
for s in shapes:
    shape_offset[s.id()] = offset_counter
    offset_counter += s.primitive_count()

    indices = dr.arange(dr.cuda.UInt, s.face_count())
    vertex_indices = dr.ravel(s.face_indices(indices), order='F')
    vertices = s.vertex_position(vertex_indices)
    vertices = vertices.torch().T.reshape(-1, 3, 3)
    face_averages.append(vertices)

face_averages = torch.vstack(face_averages)
face_averages = face_averages.sum(1) / 3

In [5]:
face_averages = mi.Point3f(face_averages.T)
face_normals = mi.Vector3f(face_normals.T)
print(face_averages)
print(face_normals)

[[-0.333333, 4.05719, 0.471405],
 [0.333333, 4.5286, 0.942809],
 [1, 5, 0.471405],
 .. 30 skipped ..,
 [1, -0.333333, -0.333333],
 [-0.333333, 1, 0.333333],
 [0.333333, 1, -0.333333]]
[[3.43526e-10, -0.707107, 0.707107],
 [0, -0.707107, 0.707107],
 [1, 0, 0],
 .. 30 skipped ..,
 [1, 0, 0],
 [0, 1, 0],
 [0, 1, 0]]


In [6]:
# Generate rays to be shot from each surface in 180 cone
# Currently generates 360 then filters based on dot product

from sionna.rt.utils import fibonacci_lattice, spawn_ray_from_sources
ray_per_source = 1000

sources = face_averages + (face_normals * 0.0001)
rays = spawn_ray_from_sources(fibonacci_lattice, ray_per_source, sources) 
print(rays)

norms_expanded = dr.repeat(face_normals, ray_per_source)
print(norms_expanded)
# shouldn't this be positive dot prod???
# active = dr.dot(rays.d, norms_expanded) > 0
# print(active)

Ray[
  o=[[-0.333333, 4.05712, 0.471475],
     [-0.333333, 4.05712, 0.471475],
     [-0.333333, 4.05712, 0.471475],
     [-0.333333, 4.05712, 0.471475],
     [-0.333333, 4.05712, 0.471475],
        35990 skipped ..,
     [0.333333, 1.0001, -0.333333],
     [0.333333, 1.0001, -0.333333],
     [0.333333, 1.0001, -0.333333],
     [0.333333, 1.0001, -0.333333],
     [0.333333, 1.0001, -0.333333]],
  d=[[0, 0, 1],
     [-0.0466353, -0.0427217, 0.997998],
     [0.00781566, 0.0890555, 0.995996],
     [0.0665843, -0.0868474, 0.993994],
     [-0.12437, 0.0219993, 0.991992],
        35990 skipped ..,
     [0.118513, -0.0436636, -0.991992],
     [-0.101274, -0.0414674, -0.993994],
     [0.0381213, 0.0808625, -0.995996],
     [0.0187565, -0.0604002, -0.997998],
     [-0, 0, -1]],
  maxt=[3.40282e+38],
  time=[0, 0, 0, 0, 0,    35990 skipped .., 0, 0, 0, 0, 0],
  wavelengths=[]
]
[[3.43526e-10, -0.707107, 0.707107],
 [3.43526e-10, -0.707107, 0.707107],
 [3.43526e-10, -0.707107, 0.707107],
 .. 35994

In [7]:
# temp = active.numpy()
# rays.d.numpy().T[temp][0:ray_per_source]

In [8]:
rays.o.numpy().T[0]

array([-0.33333334,  4.057121  ,  0.47147524], dtype=float32)

In [9]:
interactions = mi_scene.ray_intersect_preliminary(rays)#, active=active)
# print(interactions)
full_int = interactions.compute_surface_interaction(rays)
# print(full_int.p)

count = 0
for s in mi_scene.shapes():
    count += s.primitive_count()

# a=dr.select(interactions.is_valid() & (interactions.t > 1.5), interactions.prim_index, dr.cuda.UInt(count)).numpy()
a=dr.select(full_int.is_valid(), full_int.prim_index, dr.cuda.UInt(count)).numpy()
# interaction_shapes=dr.select(full_int.is_valid(), full_int.shape, dr.cuda.UInt32(36000)).numpy()
interaction_shapes = full_int.shape.numpy()[full_int.is_valid()]
# interaction_shapes[0].id()
print(interaction_shapes.shape)

# b=dr.select(interactions.is_valid(), interactions.t, dr.cuda.Float32(-1)).numpy()

# for i in range(len(a)):
#     print(a[i], b[i])



# for i, b in enumerate(a):
#     print(b, interactions.prim_index[i])
a.shape


(18413,)


(36000,)

In [10]:
full_valid = full_int.is_valid().numpy()
print(full_valid.shape)
hit_points = full_int.p.numpy().T[full_valid]

origins = rays.o.numpy().T[full_valid]
print(origins.shape)

scene.preview()

# This cell is lowkey pretty illegal but here's to being easier to implement
## THIS REQUIRES persist=True in call self._add_child(mesh, pmin, pmax, persist=True) in _plot_lines (preview.py)

# starts = np.array([[0,0,0],
#                    [1, 1, 1]])
# ends_indices = np.array([[5,5,5],
#                  [-100, -100, 0]])
colors = np.array([1.0, 0.0, 0.0], dtype=np.float32)
colors = np.tile(colors, (origins.shape[0], 1))
# print(colors)
# print(colors.dtype)
width = np.array(1.0)
scene._preview_widget._plot_lines(origins, hit_points, colors, width)

(36000,)
(18413, 3)


In [11]:
dir(full_int.prim_index)
collision_shape_i = full_int.prim_index.numpy()[full_valid]

np.unique(collision_shape_i)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11], dtype=uint32)

In [12]:
scene.preview()

Renderer(camera=PerspectiveCamera(aspect=1.31, children=(DirectionalLight(intensity=0.25, matrixWorldNeedsUpda…

In [13]:
sets = [set() for _ in range(count)]

for i in range(len(a)):
    vis = a[i]
    if vis < count:
        cur_shape = interaction_shapes[i] - 1
        print(cur_shape)
        print(mi_scene.shapes()[cur_shape].id())
        # offset = mi_scene.shapes[collision_shape_i].id()
        sets[i // ray_per_source].add(vis)

sets
for i, s in enumerate(face_averages.numpy().T):
    print(s, sets[i])

2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
1
mesh-Cube_001
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
1
mesh-Cube_001
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
1
mesh-Cube_001
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
mesh-Cube
2
me

IndexError: index 18414 is out of bounds for axis 0 with size 18413

In [ ]:
face_averages


[[-0.333333, 4.05719, 0.471405],
 [0.333333, 4.5286, 0.942809],
 [1, 5, 0.471405],
 .. 30 skipped ..,
 [1, -0.333333, -0.333333],
 [-0.333333, 1, 0.333333],
 [0.333333, 1, -0.333333]]

In [ ]:
sizes = [len(s) for s in sets]
face_averages_np = face_averages.numpy().T
starts = np.repeat(face_averages_np, repeats=sizes, axis=0)
print(starts.shape)

ends_indices = []
for s in sets:
    for shape in s:
        ends_indices.append(shape)

ends_indices = np.array(ends_indices)
ends = face_averages_np[ends_indices]
ends.shape

print(ends)

(121, 3)
[[-0.33333334  4.0571914   0.47140452]
 [ 1.          5.          0.47140452]
 [ 1.          5.         -0.47140452]
 [-1.          4.528596    0.        ]
 [-1.          5.4714046   0.        ]
 [ 0.33333334  4.528596    0.94280905]
 [ 1.          5.          0.47140452]
 [ 1.          5.         -0.47140452]
 [-1.          4.528596    0.        ]
 [-1.          5.4714046   0.        ]
 [ 1.          5.          0.47140452]
 [ 1.          5.         -0.47140452]
 [ 1.          5.          0.47140452]
 [ 1.          5.         -0.47140452]
 [ 0.33333334  4.0571914  -0.47140452]
 [-1.          4.528596    0.        ]
 [-1.          5.4714046   0.        ]
 [ 1.          5.          0.47140452]
 [ 1.          5.         -0.47140452]
 [-0.33333334  4.528596   -0.94280905]
 [ 0.33333334  5.4714046  -0.94280905]
 [-1.          4.528596    0.        ]
 [-1.          5.4714046   0.        ]
 [-0.33333334  5.942809   -0.47140452]
 [ 0.33333334  5.4714046  -0.94280905]
 [-0.33333334  5

In [ ]:
scene.preview()

# This cell is lowkey pretty illegal but here's to being easier to implement
## THIS REQUIRES persist=True in call self._add_child(mesh, pmin, pmax, persist=True) in _plot_lines (preview.py)

# starts = np.array([[0,0,0],
#                    [1, 1, 1]])
# ends_indices = np.array([[5,5,5],
#                  [-100, -100, 0]])
colors = np.array([1.0, 0.0, 0.0], dtype=np.float32)
colors = np.tile(colors, (starts.shape[0], 1))
# print(colors)
# print(colors.dtype)
width = np.array(1.0)
scene._preview_widget._plot_lines(starts, ends, colors, width)

In [ ]:
scene.preview()